## Spotify data ETL project

### Imports

In [27]:
import pandas as pd
import sqlalchemy
from sqlalchemy.orm import sessionmaker
import requests
import json
import datetime
from datetime import datetime, timedelta
import sqlite3

In [28]:
# constants
DB_LOCATION = 'sqlite:///spoty_songs.sqlite'
USER_ID = 'Hristiyan'
SPOTY_TOKEN = 'BQAO4y2euytWwn9p60ynLx-JEO0aUVvm-nIkV3zs7_EMXx6yAFUyXh0YI1THqVbrxKyvynASbQ_EmVgKiAtdvWyQsXCmC5dbxCITEI5VP-aH05-a_n05YfXxx2CvfPC__v4Q0Xep7bpkfKEb1yD7XobOTq_djRSka9HgZxk6q2jcdcetIPEbjP7yqmTDra1nMLXR9A'

In [29]:
# check if current script is being run as main program
if __name__ == "__main__":
    # create the headers for the request
    headers = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : "Bearer {token}".format(token=SPOTY_TOKEN)
    }

In [30]:
# define yesterday unix timestamp step by step
today = datetime.now()
yesterday = today - timedelta(days=1)
yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

In [31]:
# request the data from the API
r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time=yesterday_unix_timestamp), headers=headers)